<a href="https://colab.research.google.com/github/TassyaRM/MachineLearning_TK-44-G4_/blob/main/Mgg7/XGBOOST_MELB_HOUSE_PRICE_Tassya.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#NAMA : TASSYA RAMADHANTI
#NIM  : 1103204016
#XGBOOST MELB HOUSE PRICE
---

In [1]:
!pip install kaggle

In [2]:
from google.colab import files

# Upload file kaggle.json
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"tassyaramadhanti","key":"3182cbc2ed662c2e7a9035299a970b54"}'}

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [11]:
!kaggle datasets list -s "Melbourne Housing Snapshot"

ref                                       title                         size  lastUpdated          downloadCount  voteCount  usabilityRating  
----------------------------------------  ---------------------------  -----  -------------------  -------------  ---------  ---------------  
dansbecker/melbourne-housing-snapshot     Melbourne Housing Snapshot   451KB  2018-06-05 12:52:24         129820       1386  0.7058824        
ganjedi/melbourne-housing-snapshot        Melbourne Housing Snapshot   614KB  2018-01-23 19:39:49            597         10  0.4117647        
dhirajbembade/melbourne-housing-snapshot   Melbourne Housing Snapshot  451KB  2023-06-05 07:10:55             46         13  0.5294118        
danghle/melbourne-housing-snapshot        Melbourne Housing Snapshot   614KB  2018-01-19 00:34:27            110          1  0.4117647        


In [12]:
# Download dan ekstrak dataset, dimana nantinya akan satu direktori dengan notebook ini
!kaggle datasets download dhirajbembade/melbourne-housing-snapshot --unzip

  0% 0.00/451k [00:00<?, ?B/s]
100% 451k/451k [00:00<00:00, 92.6MB/s]


In [40]:
# import library

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

In [50]:
# load data

data = pd.read_csv("melb_data.csv")
data

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.79960,144.99840,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.80790,144.99340,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.80930,144.99440,Northern Metropolitan,4019.0
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,...,2.0,1.0,94.0,NaN,NaN,Yarra,-37.79690,144.99690,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,...,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.80720,144.99410,Northern Metropolitan,4019.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13575,Wheelers Hill,12 Strada Cr,4,h,1245000.0,S,Barry,26/08/2017,16.7,3150.0,...,2.0,2.0,652.0,NaN,1981.0,NaN,-37.90562,145.16761,South-Eastern Metropolitan,7392.0
13576,Williamstown,77 Merrett Dr,3,h,1031000.0,SP,Williams,26/08/2017,6.8,3016.0,...,2.0,2.0,333.0,133.0,1995.0,NaN,-37.85927,144.87904,Western Metropolitan,6380.0
13577,Williamstown,83 Power St,3,h,1170000.0,S,Raine,26/08/2017,6.8,3016.0,...,2.0,4.0,436.0,NaN,1997.0,NaN,-37.85274,144.88738,Western Metropolitan,6380.0
13578,Williamstown,96 Verdon St,4,h,2500000.0,PI,Sweeney,26/08/2017,6.8,3016.0,...,1.0,5.0,866.0,157.0,1920.0,NaN,-37.85908,144.89299,Western Metropolitan,6380.0


In [51]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Suburb         13580 non-null  object 
 1   Address        13580 non-null  object 
 2   Rooms          13580 non-null  int64  
 3   Type           13580 non-null  object 
 4   Price          13580 non-null  float64
 5   Method         13580 non-null  object 
 6   SellerG        13580 non-null  object 
 7   Date           13580 non-null  object 
 8   Distance       13580 non-null  float64
 9   Postcode       13580 non-null  float64
 10  Bedroom2       13580 non-null  float64
 11  Bathroom       13580 non-null  float64
 12  Car            13518 non-null  float64
 13  Landsize       13580 non-null  float64
 14  BuildingArea   7130 non-null   float64
 15  YearBuilt      8205 non-null   float64
 16  CouncilArea    12211 non-null  object 
 17  Lattitude      13580 non-null  float64
 18  Longti

In [52]:
# Periksa jumlah data yang hilang (NaN) dalam setiap kolom
missing_data = data.isnull().sum()

# Tampilkan data yang hilang
print(missing_data)

Suburb              0
Address             0
Rooms               0
Type                0
Price               0
Method              0
SellerG             0
Date                0
Distance            0
Postcode            0
Bedroom2            0
Bathroom            0
Car                62
Landsize            0
BuildingArea     6450
YearBuilt        5375
CouncilArea      1369
Lattitude           0
Longtitude          0
Regionname          0
Propertycount       0
dtype: int64


In [53]:
# Pemrosesan Data
data = data.dropna()  # Hapus baris dengan data yang hilang

In [55]:
data.shape

(6196, 21)

In [70]:
data = data.copy()  # Membuat salinan DataFrame
data['Bedroom2'] = data['Bedroom2'].astype(int)
data['Bathroom'] = data['Bathroom'].astype(int)
data['Car'] = data['Car'].astype(int)

In [71]:
# Select subset of predictors
cols_to_use = ['Rooms', 'Distance', 'Landsize', 'BuildingArea', 'YearBuilt']
X = data[cols_to_use]

In [72]:
# Select target
y = data.Price

# Separate data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state=42,test_size=0.3)

In [73]:
from xgboost import XGBRegressor

my_model = XGBRegressor()
my_model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [74]:
predictions = my_model.predict(X_valid)
print("Mean Absolute Errors : " + str(mean_absolute_error(predictions, y_valid)))

Mean Absolute Errors : 222286.78807490587


#Parameter Tuning
---

In [75]:
my_model_0 = XGBRegressor(n_estimators=500)
my_model_0.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=500, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [76]:
predictions_0 = my_model_0.predict(X_valid)
print("Mean Absolute Erros : " + str(mean_absolute_error(predictions_0,y_valid)))

Mean Absolute Erros : 232849.6255337211


In [77]:
my_model_1 = XGBRegressor(n_estimators=110)
my_model_1.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=110, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [78]:
# Tentukan range nilai n_estimators yang akan diuji
n_estimators_list = [100, 150, 200, 250, 300]

# Inisialisasi sebuah list untuk menyimpan hasil MAE
mae_results = []

# Loop melalui setiap nilai n_estimators
for n_estimators in n_estimators_list:
    # Inisialisasi model dengan n_estimators tertentu
    model_grid = XGBRegressor(n_estimators=n_estimators)

    # Fitting model menggunakan data training
    model_grid.fit(X_train, y_train)

    # Lakukan prediksi pada data validasi
    predictions = model_grid.predict(X_valid)

    # Hitung Mean Absolute Error
    mae = mean_absolute_error(predictions, y_valid)

    # Tampilkan nilai n_estimators dan MAE
    print(f"n_estimators: {n_estimators}, Mean Absolute Error: {mae}")

    # Tambahkan nilai MAE ke dalam list
    mae_results.append(mae)

n_estimators: 100, Mean Absolute Error: 222286.78807490587
n_estimators: 150, Mean Absolute Error: 224927.00370242738
n_estimators: 200, Mean Absolute Error: 226814.38409006858
n_estimators: 250, Mean Absolute Error: 228181.87653812533
n_estimators: 300, Mean Absolute Error: 229741.71355987762


In [79]:
my_model_2 = XGBRegressor(n_estimators=100)
my_model_2.fit(X_train, y_train,
               early_stopping_rounds=5,
               eval_set=[(X_valid, y_valid)],
               verbose=False)

predictions_2 = my_model_2.predict(X_valid)
print("Mean Absolute Error: " + str(mean_absolute_error(predictions_2, y_valid)))

Mean Absolute Error: 228898.31270172136


/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


In [80]:
my_model_1 = XGBRegressor(n_estimators=100)
my_model_1.fit(X_train, y_train)
predictions_1 = my_model_1.predict(X_valid)
print("Mean Absolute Error: " + str(mean_absolute_error(predictions_1, y_valid)))

Mean Absolute Error: 222286.78807490587


In [81]:
my_model_3 = XGBRegressor(n_estimators=100,learning_rate=0.57,n_jobs=4)
my_model_3.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.57, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=4,
             num_parallel_tree=None, random_state=None, ...)

In [82]:
predictions_3 = my_model_3.predict(X_valid)
print("Mean Absolute Error: " + str(mean_absolute_error(predictions_3, y_valid)))

Mean Absolute Error: 237989.1527366864
